# Modifying Rates

Sometimes we want to change the nuclei involved in rates to simplify our network.  Currently,
pynucastro supports changing the products.  Here's an example.

In [1]:
import pynucastro as pyna

In [2]:
reaclib_library = pyna.ReacLibLibrary()

We want to model ${}^{12}\mathrm{C} + {}^{12}\mathrm{C}$ reactions.  There are 3 rates involved.

In [3]:
filter = pyna.RateFilter(reactants=["c12", "c12"])
mylib = reaclib_library.filter(filter)
mylib                       

C12 + C12 ⟶ He4 + Ne20         [Q =   4.62 MeV] (C12 + C12 --> He4 + Ne20 <cf88_reaclib__>)
C12 + C12 ⟶ p + Na23           [Q =   2.24 MeV] (C12 + C12 --> p + Na23 <cf88_reaclib__>)
C12 + C12 ⟶ n + Mg23           [Q =  -2.60 MeV] (C12 + C12 --> n + Mg23 <cf88_reaclib__reverse>)

The rate ${}^{12}\mathrm{C}({}^{12}\mathrm{C},n){}^{23}\mathrm{Mg}$ is quickly followed by ${}^{23}\mathrm{Mg}(n,\gamma){}^{24}\mathrm{Mg}$, so we want to modify that rate sequence to just be ${}^{12}\mathrm{C}({}^{12}\mathrm{C},\gamma){}^{24}\mathrm{Mg}$

In [4]:
r = mylib.get_rate("c12 + c12 --> n + mg23 <cf88_reaclib__reverse>")
r

LookupError: rate identifier 'c12 + c12 --> n + mg23 <cf88_reaclib__reverse>' does not match a rate in this library.

This has the Q value:

In [5]:
r.Q

NameError: name 'r' is not defined

Now we modify it

In [6]:
r.modify_products("mg24")
r

NameError: name 'r' is not defined

and we see that the Q value has been updated to reflect the new endpoint

In [7]:
r.Q

NameError: name 'r' is not defined

Now let's build a network that includes the nuclei involved in our carbon burning.  We'll start by leaving off the ${}^{23}\mathrm{Mg}$

In [8]:
mylib2 = reaclib_library.linking_nuclei(["p", "he4", "c12", "o16", "ne20", "na23", "mg24"])

Now we add in our modified rate

In [9]:
mylib2 += pyna.Library(rates=[r])

NameError: name 'r' is not defined

In [10]:
mylib2

He4 + He4 + He4 ⟶ C12 + 𝛾      [Q =   7.28 MeV] (He4 + He4 + He4 --> C12 <fy05_reaclib__>)
C12 + He4 ⟶ O16 + 𝛾            [Q =   7.16 MeV] (C12 + He4 --> O16 <nac2_reaclib__>)
C12 + C12 ⟶ He4 + Ne20         [Q =   4.62 MeV] (C12 + C12 --> He4 + Ne20 <cf88_reaclib__>)
C12 + C12 ⟶ p + Na23           [Q =   2.24 MeV] (C12 + C12 --> p + Na23 <cf88_reaclib__>)
O16 + He4 ⟶ Ne20 + 𝛾           [Q =   4.73 MeV] (O16 + He4 --> Ne20 <co10_reaclib__>)
O16 + C12 ⟶ He4 + Mg24         [Q =   6.77 MeV] (O16 + C12 --> He4 + Mg24 <cf88_reaclib__>)
Ne20 + He4 ⟶ Mg24 + 𝛾          [Q =   9.32 MeV] (Ne20 + He4 --> Mg24 <il10_reaclib__>)
Na23 + p ⟶ He4 + Ne20          [Q =   2.38 MeV] (Na23 + p --> He4 + Ne20 <il10_reaclib__>)
Na23 + p ⟶ Mg24 + 𝛾            [Q =  11.69 MeV] (Na23 + p --> Mg24 <il10_reaclib__>)
C12 ⟶ He4 + He4 + He4          [Q =  -7.28 MeV] (C12 --> He4 + He4 + He4 <fy05_reaclib__reverse>)
O16 ⟶ He4 + C12                [Q =  -7.16 MeV] (O16 --> He4 + C12 <nac2_reaclib__reverse>)
Ne20 + He4 

In [11]:
rc = pyna.RateCollection(libraries=[mylib2])
fig = rc.plot(rotated=True, curved_edges=True, hide_xalpha=True)

TypeError: argument of type 'NoneType' is not iterable